In [11]:
# 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import asyncio
import json
import nest_asyncio
import pandas as pd
import re
from tqdm import tqdm
nest_asyncio.apply()

In [3]:
urls = pd.read_csv("../data/naver_pop_href.csv")

In [4]:
test = urls.iloc[:, 1:3]
test

,0
0,https://n.news.naver.com/article/008/000452225...
1,https://n.news.naver.com/article/008/000452229...
2,https://n.news.naver.com/article/008/000452228...
3,https://n.news.naver.com/article/008/000452224...
4,https://n.news.naver.com/article/008/000452241...
...,...
8335,https://n.news.naver.com/article/437/000033360...
8336,https://n.news.naver.com/article/437/000033366...
8337,https://n.news.naver.com/article/437/000033366...
8338,https://n.news.naver.com/article/437/000033369...


In [5]:
# 함수 정의
def get_comments(refer_url, comment_url) : # 댓글 목록을 json 형태로 받아오는 함수
    comments = []
    next = None
    # 처음엔 댓글 개수를 모르므로 충분히 큰 수를 넣어 줌
    comment_count = 10e6
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Safari/605.1.15',
        'referer': refer_url
    }
    
    # 수집한 댓글 수가 첫번째에 수집한 총 댓글 수 보다 많다면 반복을 종료합니다.
    while len(comments) < comment_count :
        comment_url_next = comment_url + '&moreParam.next=' + next if next else comment_url
        res = requests.get(comment_url_next, headers=headers)
        dic = json.loads(res.text[res.text.index('(')+1:-2])
        comments.extend(list(map(lambda x : {
            'url' : refer_url,
            'text': x['contents'],
            'reply_count' : x['replyCount'], 
            'uid': x['idNo'],
            'uname' : x['userName'],
            'like': x['sympathyCount'], 
            'dislike': x['antipathyCount'],
            'c_time' : x['modTime'],
            'cid': x['commentNo'],
            'pid' : x['parentCommentNo'] 
            }, dic['result']['commentList'])))
        comment_count = dic['result']['count']['comment']
        next = dic['result']['morePage']['next'] if comment_count else None
    # 필터로 삭제된 댓글을 걸러줍니다
    comments=list(filter(lambda x: len(x['text']), comments))
    return comments



In [6]:
def get_data(oid, aid) :
    try :
        # refer_url: 댓글 보기를 누르면 나오는 댓글 페이지 주소
        # comment_url: 네트워크 탭에서 확인 가능한 동적으로 생성되는 주소
        refer_url = f'https://n.news.naver.com/mnews/article/comment/{oid}/{aid}?sid=100'
        comment_url = f'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_politics_m3&pool=cbox5&_cv=20220623185938&_callback=jQuery33103200637113167857_1656171100524&lang=ko&country=KR&objectId=news{oid}%2C{aid}&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=20&followSize=100&sort=new&includeAllStatus=true&_=1656171100525'

        comments = get_comments(refer_url, comment_url)
        comments = pd.DataFrame(comments)
        # 댓글 수가 0개인 기사를 어떻게 할지 추후에 결정해야 함
        return comments
    except :
        return None


In [ ]:
df = pd.DataFrame()
for url in tqdm(test['0']):
    try:
        oid = re.findall('article/([^\*]*)/', str(url))[0]
        aid = re.findall(f'{oid}/([^\*]*)[?]', str(url))[0]
        sid = url[-3:]
        temp = get_data(oid, aid)
        temp['sid'] = sid
        df =pd.concat([df, temp])
    except:
        print(url)

 12%|█▏        | 1042/8340 [45:56<6:52:19,  3.39s/it] 

In [ ]:
df.reset_index(drop = True, inplace = True)
df

In [ ]:
df.to_pickle('../data/popular_news.pkl')
df.to_csv('../data/popular_news.csv')

In [12]:
df = pd.read_csv("../data.popular_news.csv")

In [15]:
df[['text', 'c_time', 'url']].to_csv("./sample_comments.csv")